In [3]:
from pycocotools.coco import COCO
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
import json
from nltk.corpus import stopwords
import re
import string
import numpy as np
tqdm.pandas()

c:\users\peter\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
path = "../data/2017/captions.json"
data = json.load(open(path, 'r'))
df = pd.DataFrame.from_records(data['images'])
df.drop(columns=['file_name', 'license', 'date_captured', 'flickr_url'], inplace=True)
print(len(df))
df.head()

5000


,coco_url,height,width,id
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482


In [5]:
def get_captions(image_id):
    captions = coco_captions.loadAnns(coco_captions.getAnnIds(imgIds=image_id))
    captions = ' '.join(x['caption'] for x in captions)
    return captions

In [6]:
coco_captions = COCO(path)
df['captions'] = df['id'].progress_apply(lambda x: get_captions(x))

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [7]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,A man is in a kitchen making pizzas. Man in ap...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,The dining table near the kitchen has a bowl o...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,a person with a shopping cart on a city street...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,A person on a skateboard and bike at a skate p...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,a blue bike parked on a side walk A bicycle i...


In [8]:
cond1 = df['height'] < 0.8*df['width']
cond2 = df['height'] > 0.6*df['width']
df = df[cond1 & cond2]

In [9]:
print(len(df))
df.head()

3242


,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,A man is in a kitchen making pizzas. Man in ap...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,The dining table near the kitchen has a bowl o...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,a person with a shopping cart on a city street...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,A person on a skateboard and bike at a skate p...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,a blue bike parked on a side walk A bicycle i...


In [33]:
def add_stuff_to_captions(caption, image_id):
    anns = coco_captions.loadAnns(coco_captions.getAnnIds(imgIds=image_id))
    
    stuff = []
    for ann in anns:
        key = ann['category_id']
        try:
            value = int_to_cat[key]
            words = re.findall(r"[\w']+", value)
            if 'other' in words:
                words.remove('other')
            stuff.extend(words)
        except KeyError:
            continue
    stuff = ' '.join(stuff)
    caption = f'{stuff} {caption}'
    return caption

In [34]:
path = "../data/2017/stuff.json"
coco_captions = COCO(path)
int_to_cat = np.load('../data/2017/int_to_cat.npy', allow_pickle=True).item()
f = lambda row: add_stuff_to_captions(row['captions'], row['id'])
df['captions'] = df[['captions', 'id']].progress_apply(f, axis=1)

loading annotations into memory...
Done (t=0.55s)
creating index...
index created!


In [37]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,branch cabinet floor floor stone food furnitur...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,cabinet cardboard ceiling door stuff floor mar...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,building clothes metal pavement plastic wall b...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,building fog pavement platform wall concrete b...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,building dirt metal pavement road tree buildin...


In [38]:
df['captions'] = df['captions'].str.lower()

In [39]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,branch cabinet floor floor stone food furnitur...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,cabinet cardboard ceiling door stuff floor mar...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,building clothes metal pavement plastic wall b...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,building fog pavement platform wall concrete b...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,building dirt metal pavement road tree buildin...


In [40]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['captions'] = df['captions'].progress_apply(lambda text: remove_punctuation(text))

In [41]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,branch cabinet floor floor stone food furnitur...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,cabinet cardboard ceiling door stuff floor mar...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,building clothes metal pavement plastic wall b...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,building fog pavement platform wall concrete b...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,building dirt metal pavement road tree buildin...


In [42]:
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    return ' '.join([word for word in str(text).split() if word not in stop_words])

df['captions'] = df['captions'].progress_apply(lambda text: remove_stop_words(text))

In [43]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,branch cabinet floor floor stone food furnitur...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,cabinet cardboard ceiling door stuff floor mar...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,building clothes metal pavement plastic wall b...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,building fog pavement platform wall concrete b...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,building dirt metal pavement road tree buildin...


In [47]:
def remove_duplicates(text):
    words = str(text).split()
    unique_words = set(words)
    return ' '.join(list(unique_words))

df['captions'] = df['captions'].progress_apply(lambda text: remove_duplicates(text))

In [49]:
df.head()

,coco_url,height,width,id,captions
0,http://images.cocodataset.org/val2017/00000039...,427,640,397133,making branch food dough wall pans pots near b...
1,http://images.cocodataset.org/val2017/00000003...,230,352,37777,ceiling wall near us plastic floor small see t...
2,http://images.cocodataset.org/val2017/00000025...,428,640,252219,building city people cart cup walking street w...
3,http://images.cocodataset.org/val2017/00000008...,480,640,87038,building platform wall park performs jumps fal...
4,http://images.cocodataset.org/val2017/00000017...,388,640,174482,building city tree chained street near blue po...


In [50]:
df.to_csv('../data/2017/captions.csv', index=False)